# Groundedness Checking Skills

In this notebook we run a simple grounding pipeline, to see if a summary text has any ungrounded additions as compared to the original.

Let us first define our grounding text:

In [1]:
grounding_text = """I am by birth a Genevese, and my family is one of the most distinguished of that republic.
My ancestors had been for many years counsellors and syndics, and my father had filled several public situations
with honour and reputation. He was respected by all who knew him for his integrity and indefatigable attention
to public business. He passed his younger days perpetually occupied by the affairs of his country; a variety
of circumstances had prevented his marrying early, nor was it until the decline of life that he became a husband
and the father of a family.

As the circumstances of his marriage illustrate his character, I cannot refrain from relating them. One of his
most intimate friends was a merchant who, from a flourishing state, fell, through numerous mischances, into poverty.
This man, whose name was Beaufort, was of a proud and unbending disposition and could not bear to live in poverty
and oblivion in the same country where he had formerly been distinguished for his rank and magnificence. Having
paid his debts, therefore, in the most honourable manner, he retreated with his daughter to the town of Lucerne,
where he lived unknown and in wretchedness. My father loved Beaufort with the truest friendship and was deeply
grieved by his retreat in these unfortunate circumstances. He bitterly deplored the false pride which led his friend
to a conduct so little worthy of the affection that united them. He lost no time in endeavouring to seek him out,
with the hope of persuading him to begin the world again through his credit and assistance.

Beaufort had taken effectual measures to conceal himself, and it was ten months before my father discovered his
abode. Overjoyed at this discovery, he hastened to the house, which was situated in a mean street near the Reuss.
But when he entered, misery and despair alone welcomed him. Beaufort had saved but a very small sum of money from
the wreck of his fortunes, but it was sufficient to provide him with sustenance for some months, and in the meantime
he hoped to procure some respectable employment in a merchant's house. The interval was, consequently, spent in
inaction; his grief only became more deep and rankling when he had leisure for reflection, and at length it took
so fast hold of his mind that at the end of three months he lay on a bed of sickness, incapable of any exertion.

His daughter attended him with the greatest tenderness, but she saw with despair that their little fund was
rapidly decreasing and that there was no other prospect of support. But Caroline Beaufort possessed a mind of an
uncommon mould, and her courage rose to support her in her adversity. She procured plain work; she plaited straw
and by various means contrived to earn a pittance scarcely sufficient to support life.

Several months passed in this manner. Her father grew worse; her time was more entirely occupied in attending him;
her means of subsistence decreased; and in the tenth month her father died in her arms, leaving her an orphan and
a beggar. This last blow overcame her, and she knelt by Beaufort's coffin weeping bitterly, when my father entered
the chamber. He came like a protecting spirit to the poor girl, who committed herself to his care; and after the
interment of his friend he conducted her to Geneva and placed her under the protection of a relation. Two years
after this event Caroline became his wife."""

## Set up Semantic Kernel

We prepare our kernel in the usual way:

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

## Import the Skills

We are going to be using the grounding skill, to check its quality, and remove ungrounded additions:

In [3]:
from semantic_kernel.core_skills.text_skill import TextSkill

# note: using skills from the samples folder
skills_directory = "../../skills"

groundingSemanticFunctions = kernel.import_semantic_skill_from_directory(skills_directory, "GroundingSkill")

We can also extract the individual semantic functions for our use:

In [4]:
entity_extraction = groundingSemanticFunctions["ExtractEntities"]
reference_check = groundingSemanticFunctions["ReferenceCheckEntities"]
entity_excision = groundingSemanticFunctions["ExciseEntities"]

## Calling Individual Semantic Functions

We will start by calling the individual grounding functions in turn, to show their use. For this we need to create a same summary text:

In [5]:
summary_text = """
My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of
misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out,
finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and
his daughter, Mary. Mary procured work to eek out a living, but after ten months her father died, leaving
her a beggar. My father came to her aid and two years later they married when they visited Rome.
"""

summary_text = summary_text.replace("\n", " ").replace("  ", " ")
print(summary_text)

 My father, a respected resident of Milan, was a close friend of a merchant named Beaufort who, after a series of misfortunes, moved to Zurich in poverty. My father was upset by his friend's troubles and sought him out, finding him in a mean street. Beaufort had saved a small sum of money, but it was not enough to support him and his daughter, Mary. Mary procured work to eek out a living, but after ten months her father died, leaving her a beggar. My father came to her aid and two years later they married when they visited Rome. 


Some things to note:

- The implied residence of Geneva has been changed to Milan
- Lucerne has been changed to Zurich
- Caroline has been renamed as Mary
- A reference to Rome has been added


The grounding skill has three stages:

1. Extract entities from a summary text
2. Perform a reference check against the grounding text
3. Excise any entities which failed the reference check from the summary

Now, let us start calling individual semantic functions.

### Preparing the Context

Semantic functions operate in a context, which provides extra parameters for their operation. For the grounding skill, the context is expected to supply the topic for the reference checking, and some particular examples of them:

In [6]:
context = kernel.create_new_context()
context["topic"] = "people and places"
context["example_entities"] = "John, Jane, mother, brother, Paris, Rome"

### Extracting the Entities

The first function we need is entity extraction. We are going to take our summary text, and get a list of entities found within it. For this we use `entity_extraction()`:

In [7]:
extraction_result = entity_extraction(summary_text, context=context)

print(extraction_result.result)

render_async: # Task Description

1. Please extract a list of entities related to people and places from the text between the <input_context> tags.
2. These are some sample entities related to people and places to help you decide what to extract: John, Jane, mother, brother, Paris, Rome
3. The list in (2) is provided to help you decide which entities to extract, but you may choose to include entities which are related to people and places but which are not listed in (2).
4. As the first part of your response, generate a bulleted list of each of the items in (1) together with an explanation of what they are.
5. Go over each item in your bulleted list and read the explanation of what it is. Keep items which are related to people and places
6. Go over each item in your bulleted list and verify that it appears between the <input_context> tags.
7. Go over each item in your bulleted list and check for duplicates. Keep only one example of each. Duplicates may be:
    - Abbreviations
    - Reu

So we have our list of entities in the summary

### Performing the reference check

We now use the grounding text to see if the entities we found are grounded. We start by adding the grounding text to our context:

In [8]:
context["reference_context"] = grounding_text

With this in place, we can run the reference checking function. This will use both the `reference_context` and the `entities` from the context object:

In [9]:
grounding_result = reference_check(extraction_result.result, context=context)

print(grounding_result.result)

render_async: # Task Description

1. Go over each item in the list between the <entities> tags and for each item, read through the data between the <grounding_context> tags and determine if each item is grounded in any of the data between the <grounding_context> tags. Be sure to consider all of the reference items.
2. When looking for references to the items in (1) look for re-phrasings, alternate names or equivalent meanings in the context in addition to exact matches
3. Create a bulleted list of the items in (1) together with an explanation of whether or not they were referred to in the context, making sure to consider step (2) where you note down references in the form of re-phrasings, alternate names or equivalent meanings in the context, as well as exact matches.
4. Split the list into two sub-lists, those items which are referenced in the <grounding_context> (these are 'grounded') and those which are not (these are 'ungrounded').
5. Make one last pass over the two lists from (4) 

So we now have a list of ungrounded entities (of course, this list may not be well grounded itself). Let us store this in the context:

In [ ]:
context["ungrounded_entities"] = grounding_result.result

### Excising the ungrounded entities

Finally we can remove the ungrounded entities from the text:

In [ ]:
excision_result = entity_excision(summary_text, context=context)

print(excision_result.result)

In [ ]:
type(entity_excision._function)